<a href="https://colab.research.google.com/github/justinzw/cortx-1/blob/640b7e6ddf/beginner_ipynb_with_CORTX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners with CORTX

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [ ]:
import tensorflow as tf

Use boto3 to connect CORTX and download the MNIST dataset:

In [ ]:
pip install boto3

In [ ]:
import boto3
from botocore.client import Config

s3 = boto3.resource('s3',
                    endpoint_url='http://endpoint.com',
                    aws_access_key_id='<ACCESS_KEY>',
                    aws_secret_access_key='<SECRET_ACCESS_KEY>')

In [ ]:
s3.Bucket('tensorflow').download_file('mnist.npz', '/tmp/CORTX_mnist.npz')

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data(path='/tmp/CORTX_mnist.npz')
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.05518755,  0.3409231 , -0.24164324,  0.2247566 , -0.12021304,
        -0.01516739,  0.50523347,  0.18401378,  0.19939108, -0.3036701 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.08540873, 0.12692025, 0.07088031, 0.11300053, 0.0800317 ,
        0.08889612, 0.14958568, 0.10848909, 0.11017026, 0.0666174 ]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

2.420287

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2950 - accuracy: 0.9159
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1400 - accuracy: 0.9582
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1050 - accuracy: 0.9685
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0858 - accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0736 - accuracy: 0.9773


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0730 - accuracy: 0.9772


[0.07296022772789001, 0.9771999716758728]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.2460111e-07, 3.3192830e-08, 7.4136151e-05, 2.8721677e-04,
        2.7533427e-11, 1.6872079e-08, 3.0393334e-13, 9.9963391e-01,
        4.4962437e-08, 4.4430180e-06],
       [5.1186837e-09, 1.1393359e-05, 9.9998260e-01, 6.0325006e-06,
        5.0443937e-13, 2.7553659e-10, 1.4005291e-09, 2.3628338e-14,
        1.2757444e-08, 2.7531869e-13],
       [5.1087472e-08, 9.9788100e-01, 1.7354591e-04, 4.0391384e-05,
        2.0346521e-04, 7.9386073e-06, 1.6304833e-05, 1.4846858e-03,
        1.9139599e-04, 1.2502475e-06],
       [9.9955064e-01, 2.8826777e-09, 1.2996538e-04, 1.8431102e-08,
        1.7917264e-05, 1.8262242e-06, 2.3946824e-04, 4.0273571e-05,
        2.9418024e-07, 1.9773055e-05],
       [9.1059737e-06, 1.1871875e-09, 2.0305688e-05, 2.4578776e-08,
        9.9809712e-01, 4.1535947e-07, 1.7751605e-05, 4.5101249e-05,
        4.6989655e-07, 1.8095631e-03]], dtype=float32)>